In [48]:
"""
=============================================================
STEP 1 — DOWNLOAD AMAZON SALES DATASET USING KAGGLEHUB
=============================================================
This block downloads the dataset from Kaggle and returns the 
local path where the files are stored. KaggleHub simplifies the 
entire download process by handling authentication and caching.
"""

import kagglehub

#Downloading The Data Set from Kaggle and getting the path
data_set_path = kagglehub.dataset_download("ikramshah512/amazon-products-sales-dataset-42k-items-2025")

print("Path to data_set:", data_set_path)


Path to data_set: /Users/slsabilhassan/.cache/kagglehub/datasets/ikramshah512/amazon-products-sales-dataset-42k-items-2025/versions/1


In [18]:
"""
=============================================================
STEP 2 — INSPECT DOWNLOADED FILES
=============================================================
We list all files inside the dataset directory returned by KaggleHub.
This helps verify what raw and cleaned data files are available.
"""

import pandas as pd
import numpy as np
import os
#Checking what we have in our file
print("Dataset files:")
for f in os.listdir(data_set_path):
    print(f)


Dataset files:
amazon_products_sales_data_cleaned.csv
amazon_products_sales_data_uncleaned.csv


In [30]:
"""
=============================================================
STEP 3 — LOAD CLEANED AMAZON PRODUCT DATA
=============================================================
We construct the full path to the cleaned CSV file and load it
into a Pandas DataFrame for further analysis.
"""

clean_path = os.path.join(data_set_path, "amazon_products_sales_data_cleaned.csv")
df = pd.read_csv(clean_path)

df.head()


,product_title,product_rating,total_reviews,purchased_last_month,discounted_price,original_price,is_best_seller,is_sponsored,has_coupon,buy_box_availability,delivery_date,sustainability_tags,product_image_url,product_page_url,data_collected_at,product_category,discount_percentage
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300.0,89.68,159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,2025-09-01,Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Phones,43.60
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000.0,9.99,15.99,No Badge,Sponsored,No Coupon,Add to cart,2025-08-29,NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Laptops,37.52
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000.0,314.00,349.00,No Badge,Sponsored,No Coupon,Add to cart,2025-09-01,NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2025-08-21 11:14:29,Laptops,10.03
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000.0,162.24,162.24,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,https://www.amazon.com/Apple-Cancellation-Tran...,2025-08-21 11:14:29,Phones,0.00
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000.0,72.74,72.74,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,https://www.amazon.com/Apple-MX542LL-A-AirTag-...,2025-08-21 11:14:29,Phones,0.00


In [51]:
"""
=============================================================
STEP 4 - DATA CLEANING & FEATURE ENGINEERING — EXTENSIVE DOCUMENTATION
=============================================================

This block processes the raw Amazon product dataset and transforms it into a 
machine-learning-ready DataFrame. Each step is explained in detail so that the 
pipeline is transparent, reproducible, and educational.

The steps:
1. Basic Inspection
2. Cleaning Missing Data & Duplicates
3. Brand Feature Engineering
4. Text-Based Features
5. Price-Based Features
6. Review-Based Features
7. Trust & Credibility Indicators
8. Temporal/Sales Features
9. Category-Level Features
10. Interaction-Based Features
11. Outlier Handling
12. Final Feature Summary

This pipeline helps uncover:
- Pricing behaviors  
- Review patterns  
- Brand effects  
- Category competitiveness  
- Sales momentum  
- Trustworthiness signals  
"""

print("="*70)
print("DATA CLEANING & FEATURE ENGINEERING — BEGIN")
print("="*70)

# ============================================
# 1. BASIC DATA INSPECTION
# ============================================

print("\n1. INITIAL DATA INSPECTION")
print("-"*70)
print("Shape tells us the dataset size (rows × columns).")
print(f"Dataset shape: {df.shape}")

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nData types tell us which columns need conversion:")
print(df.dtypes)


DATA CLEANING & FEATURE ENGINEERING — BEGIN

1. INITIAL DATA INSPECTION
----------------------------------------------------------------------
Shape tells us the dataset size (rows × columns).
Dataset shape: (37718, 44)

Missing values per column:
product_title                  0
product_rating                 0
total_reviews                  0
purchased_last_month        9362
discounted_price               0
original_price                 0
is_best_seller                 0
is_sponsored                   0
has_coupon                     0
buy_box_availability       12055
delivery_date               9385
sustainability_tags        34660
product_image_url              0
product_page_url             131
data_collected_at              0
product_category               0
discount_percentage            0
brand                          0
title_length                   0
title_word_count               0
has_numbers_in_title           0
price_diff                     0
price_category            

In [52]:
# ============================================
# 2. BASIC CLEANING
# ============================================
print("\n📌 WHY THIS STEP MATTERS:")
print("We remove duplicates so that repeated product listings do not bias the analysis. We also drop")
print("rows missing critical fields like title, rating, or price, since ML models cannot learn meaningful")
print("patterns from incomplete entries. This ensures data reliability and integrity in downstream tasks.")
print("")
print("\n2. BASIC CLEANING")
print("-"*70)

# --- Remove duplicate products (same title + same URL) ---
original_rows = len(df)
df = df.drop_duplicates(subset=['product_title', 'product_page_url'], keep='first')
print(f"Removed {original_rows - len(df)} duplicate products")

# --- Handle missing critical values ---
required_columns = ["product_title", "product_rating", "discounted_price"]
df = df.dropna(subset=required_columns)
print(f"Shape after removing rows with missing critical values: {df.shape}")



📌 WHY THIS STEP MATTERS:
We remove duplicates so that repeated product listings do not bias the analysis. We also drop
rows missing critical fields like title, rating, or price, since ML models cannot learn meaningful
patterns from incomplete entries. This ensures data reliability and integrity in downstream tasks.


2. BASIC CLEANING
----------------------------------------------------------------------
Removed 0 duplicate products
Shape after removing rows with missing critical values: (37718, 44)


In [53]:
# ============================================
# 3. FEATURE ENGINEERING - BRAND
# ============================================
print("\n📌 WHY THIS STEP MATTERS:")
print("Extracting the brand allows us to analyze brand influence on reviews, pricing, and trust. Brands")
print("often have strong effects on customer behavior, and identifying them enables deeper insights")
print("into category competition, premium branding, and product quality perception.")
print("")
print("\n3. BRAND FEATURES")
print("-"*70)
print("Extracting brand from the product title helps analyze brand behavior.")

# Extract brand = first word in product title (very common technique)
df['brand'] = df['product_title'].str.split().str[0].str.lower()

# Remove special characters (brands shouldn't contain symbols)
df["brand"] = df["brand"].str.replace(r"[^\w\s]", "", regex=True)

# Some extracted brands are meaningless (e.g., 'for', 'set', 'with')
generic_brands = ["new", "for", "the", "pack", "set", "with", ""]
df.loc[df["brand"].isin(generic_brands), "brand"] = "generic"

print(f"Total unique brands detected: {df['brand'].nunique()}")
print("\nTop 10 brands:\n", df["brand"].value_counts().head(10))


📌 WHY THIS STEP MATTERS:
Extracting the brand allows us to analyze brand influence on reviews, pricing, and trust. Brands
often have strong effects on customer behavior, and identifying them enables deeper insights
into category competition, premium branding, and product quality perception.


3. BRAND FEATURES
----------------------------------------------------------------------
Extracting brand from the product title helps analyze brand behavior.
Total unique brands detected: 627

Top 10 brands:
 brand
asurion    2792
hp         1707
dell        903
canon       779
lenovo      744
samsung     708
ugreen      690
sony        688
avery       586
epson       573
Name: count, dtype: int64


In [55]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Product titles contain rich semantic cues—long titles may imply detailed products, numbers often")
print("signal specifications (e.g., '256GB', '2-pack'), and word count reflects information density. These")
print("text-derived features help models understand product positioning and customer expectations.")
print("")
# ============================================
# 4. FEATURE ENGINEERING - TEXT FEATURES
# ============================================
print("\n4. TEXT FEATURES")
print("-"*70)
print("These features describe the product title itself.")

# --- Title length in characters ---
df["title_length"] = df["product_title"].astype(str).apply(len)

# --- Number of words ---
df["title_word_count"] = df["product_title"].astype(str).apply(lambda x: len(x.split()))

# --- Whether the title contains numbers (indicates specs like '128GB' or '2 pack') ---
df["has_numbers_in_title"] = df["product_title"].str.contains(r"\d+", regex=True).astype(int)

print(f"Average title length: {df['title_length'].mean():.2f} characters.")
print(f"Average word count: {df['title_word_count'].mean():.2f} words.")



📌 WHY THIS STEP MATTERS:
Product titles contain rich semantic cues—long titles may imply detailed products, numbers often
signal specifications (e.g., '256GB', '2-pack'), and word count reflects information density. These
text-derived features help models understand product positioning and customer expectations.


4. TEXT FEATURES
----------------------------------------------------------------------
These features describe the product title itself.
Average title length: 131.79 characters.
Average word count: 20.44 words.


In [60]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Price-based features reveal how products are positioned in the market. Customers respond strongly")
print("to discounts, price tiers, and perceived value. Extracting price differences and discount rates helps")
print("us analyze consumer behavior and identify which price segments dominate the marketplace.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Computing price_diff to measure how much each product is discounted.")
print("- Calculating discount_percentage to understand relative discount severity.")
print("- Grouping products into price categories (budget → luxury).")
print("- Creating a binary flag indicating whether a product is discounted.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• A higher average discount may indicate a competitive marketplace or seasonal sales.")
print("• If most products fall in the 'budget' tier, the category is price-sensitive.")
print("• discount_percentage helps identify aggressive pricing strategies.")
print("• price_category distribution reveals the overall price structure of the dataset.")

# ============================================
# 5. FEATURE ENGINEERING - PRICE FEATURES
# ============================================

print("\n5. PRICE FEATURES")
print("-"*70)

# Difference between original and discounted price
df["price_diff"] = df["original_price"] - df["discounted_price"]

# Discount percentage
df["discount_percentage"] = (
    (df["price_diff"] / df["original_price"]) * 100
).replace([np.inf, -np.inf], 0).fillna(0)

# Categorize products by price range
df["price_category"] = pd.cut(
    df["discounted_price"],
    bins=[0, 20, 50, 100, np.inf],
    labels=["budget", "mid_range", "premium", "luxury"]
)

# Whether the product has any discount
df["has_discount"] = (df["price_diff"] > 0).astype(int)

print(f"Average discount: ${df['price_diff'].mean():.2f}")
print(f"Average discount percentage: {df['discount_percentage'].mean():.2f}%")
print("\nPrice category distribution:\n", df["price_category"].value_counts())


📌 WHY THIS STEP MATTERS:
Price-based features reveal how products are positioned in the market. Customers respond strongly
to discounts, price tiers, and perceived value. Extracting price differences and discount rates helps
us analyze consumer behavior and identify which price segments dominate the marketplace.

🔍 WHAT WE'RE DOING:
- Computing price_diff to measure how much each product is discounted.
- Calculating discount_percentage to understand relative discount severity.
- Grouping products into price categories (budget → luxury).
- Creating a binary flag indicating whether a product is discounted.

📈 HOW TO INTERPRET THE OUTPUT:
• A higher average discount may indicate a competitive marketplace or seasonal sales.
• If most products fall in the 'budget' tier, the category is price-sensitive.
• discount_percentage helps identify aggressive pricing strategies.
• price_category distribution reveals the overall price structure of the dataset.

5. PRICE FEATURES
---------------------

In [61]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Reviews are central to customer trust. High ratings, large review counts, and strong engagement")
print("influence conversion rates. These engineered features quantify how trusted and popular each")
print("product is, which is essential for ranking models or sales forecasting.")

print("\n🔍 WHAT WE'RE DOING:")
print("- review_score combines rating × total_reviews to capture review impact.")
print("- reviews_per_dollar shows how review intensity changes with price.")
print("- rating_category groups products into qualitative scoring bands.")
print("- high_engagement flags products in the top 25% of review counts.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• review_score shows which products dominate customer attention.")
print("• The engagement threshold tells you how many reviews qualify as 'high engagement'.")
print("• rating_category reveals quality distribution (excellent vs poor).")
print("• reviews_per_dollar highlights which items deliver strong social proof relative to price.")
print("")
# ============================================
# 6. FEATURE ENGINEERING - REVIEW FEATURES
# ============================================

print("\n6. REVIEW FEATURES")
print("-"*70)

# Composite review score = rating × number of reviews
df["review_score"] = df["product_rating"] * df["total_reviews"]

# Review density = reviews per dollar (expensive items often have fewer)
df["reviews_per_dollar"] = df["total_reviews"] / df["discounted_price"].replace(0, 1)

# Rating category
df["rating_category"] = pd.cut(
    df["product_rating"],
    bins=[0, 3.5, 4.0, 4.5, 5.0],
    labels=["poor", "average", "good", "excellent"]
)

# High engagement flag: products above 75th percentile reviews
engagement_cutoff = df["total_reviews"].quantile(0.75)
df["high_engagement"] = (df["total_reviews"] >= engagement_cutoff).astype(int)

print(f"Avg review score: {df['review_score'].mean():.2f}")
print(f"High engagement threshold: {engagement_cutoff:.0f} reviews")
print("\nRating category distribution:\n", df["rating_category"].value_counts())


📌 WHY THIS STEP MATTERS:
Reviews are central to customer trust. High ratings, large review counts, and strong engagement
influence conversion rates. These engineered features quantify how trusted and popular each
product is, which is essential for ranking models or sales forecasting.

🔍 WHAT WE'RE DOING:
- review_score combines rating × total_reviews to capture review impact.
- reviews_per_dollar shows how review intensity changes with price.
- rating_category groups products into qualitative scoring bands.
- high_engagement flags products in the top 25% of review counts.

📈 HOW TO INTERPRET THE OUTPUT:
• review_score shows which products dominate customer attention.
• The engagement threshold tells you how many reviews qualify as 'high engagement'.
• rating_category reveals quality distribution (excellent vs poor).
• reviews_per_dollar highlights which items deliver strong social proof relative to price.


6. REVIEW FEATURES
-----------------------------------------------------------

In [62]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Badges like 'Best Seller' or 'Sponsored', coupons, and sustainability tags influence visibility and")
print("perceived legitimacy. These features help quantify trust signals that affect consumer decision-making.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Converting Amazon badges into binary indicators.")
print("- Creating a composite trust_score that weighs credibility factors.")
print("- Measuring presence of coupons, sustainability tags, and buy-box availability.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• The number of Best Sellers tells us how many top-performing items Amazon flags in this dataset.")
print("• Sponsored products reveal advertising strategies within categories.")
print("• A higher average trust_score means many products have strong credibility signals.")
print("• Coupons indicate promotional activity that may boost conversions.")
print("")
# ============================================================
# 7. TRUST SIGNALS (Badges, Sponsored, Sustainability)
# ============================================================
print("\n7. TRUST SIGNALS")
print("-"*70)
print("These features help estimate product legitimacy and credibility.")

df["is_best_seller_binary"] = (df["is_best_seller"] == "Best Seller").astype(int)
df["is_sponsored_binary"] = (df["is_sponsored"] == "Sponsored").astype(int)
df["has_coupon_binary"] = df["has_coupon"].apply(lambda x: 0 if pd.isna(x) or "No Coupon" in str(x) else 1)
df["has_sustainability"] = df["sustainability_tags"].notna().astype(int)
df["has_buy_box"] = df["buy_box_availability"].notna().astype(int)

# Weighted trust score
df["trust_score"] = (
    df["is_best_seller_binary"] * 3 +
    (df["product_rating"] >= 4.5).astype(int) * 2 +
    (df["total_reviews"] >= 1000).astype(int) * 2 +
    df["has_coupon_binary"] +
    df["has_sustainability"]
)

print(f"Best sellers: {df['is_best_seller_binary'].sum()}")
print(f"Sponsored: {df['is_sponsored_binary'].sum()}")
print(f"Products with coupon: {df['has_coupon_binary'].sum()}")
print(f"Avg trust score: {df['trust_score'].mean():.2f}")


📌 WHY THIS STEP MATTERS:
Badges like 'Best Seller' or 'Sponsored', coupons, and sustainability tags influence visibility and
perceived legitimacy. These features help quantify trust signals that affect consumer decision-making.

🔍 WHAT WE'RE DOING:
- Converting Amazon badges into binary indicators.
- Creating a composite trust_score that weighs credibility factors.
- Measuring presence of coupons, sustainability tags, and buy-box availability.

📈 HOW TO INTERPRET THE OUTPUT:
• The number of Best Sellers tells us how many top-performing items Amazon flags in this dataset.
• Sponsored products reveal advertising strategies within categories.
• A higher average trust_score means many products have strong credibility signals.
• Coupons indicate promotional activity that may boost conversions.


7. TRUST SIGNALS
----------------------------------------------------------------------
These features help estimate product legitimacy and credibility.
Best sellers: 264
Sponsored: 5154
Products w

In [63]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Temporal data helps measure sales momentum, trends, and product popularity over time. These features")
print("are essential for forecasting demand, detecting viral products, and understanding seasonality.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Creating sales_velocity from purchased_last_month.")
print("- Categorizing products into momentum levels (slow → viral).")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• Average monthly purchases reveal overall dataset activity.")
print("• 'Viral' products indicate sharp demand spikes.")
print("• The distribution of momentum categories shows how competitive the marketplace is in real-time.")
print("• If this section prints 'No temporal data', the dataset lacks time-based sales features.")
print("")
# ============================================================
# 8. TEMPORAL FEATURES (if dataset includes monthly purchases)
# ============================================================
print("\n8. TEMPORAL FEATURES")
print("-"*70)

if "purchased_last_month" in df.columns:
    df["sales_velocity"] = df["purchased_last_month"].fillna(0)
    df["sales_momentum"] = pd.cut(
        df["sales_velocity"],
        bins=[0, 100, 500, 1000, np.inf],
        labels=["slow", "moderate", "fast", "viral"]
    )
    print(f"Avg monthly purchases: {df['sales_velocity'].mean():.2f}")
else:
    print("No temporal sales data available.")



📌 WHY THIS STEP MATTERS:
Temporal data helps measure sales momentum, trends, and product popularity over time. These features
are essential for forecasting demand, detecting viral products, and understanding seasonality.

🔍 WHAT WE'RE DOING:
- Creating sales_velocity from purchased_last_month.
- Categorizing products into momentum levels (slow → viral).

📈 HOW TO INTERPRET THE OUTPUT:
• Average monthly purchases reveal overall dataset activity.
• 'Viral' products indicate sharp demand spikes.
• The distribution of momentum categories shows how competitive the marketplace is in real-time.
• If this section prints 'No temporal data', the dataset lacks time-based sales features.


8. TEMPORAL FEATURES
----------------------------------------------------------------------
Avg monthly purchases: 409.08


In [64]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Category-based features help us understand product competition and how well a product performs")
print("compared to its peers. This is crucial for understanding market structure and identifying standout products.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Calculating category_competition: number of products in each category.")
print("- Computing category_avg_rating to measure category quality.")
print("- Creating rating_vs_category to show how each product ranks relative to its peers.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• The most competitive category has the highest product count — likely a saturated market.")
print("• The least competitive category may reflect niche or specialized items.")
print("• rating_vs_category > 0 means the product is above average for its category.")
print("• Large gaps indicate products that strongly overperform or underperform relative to peers.")
print("")
# ============================================================
# 9. CATEGORY FEATURES
# ============================================================
print("\n9. CATEGORY FEATURES")
print("-"*70)

# How competitive is each category?
category_counts = df["product_category"].value_counts()
df["category_competition"] = df["product_category"].map(category_counts)

# Average rating in category
df["category_avg_rating"] = df.groupby("product_category")["product_rating"].transform("mean")

# Difference between product rating and category avg
df["rating_vs_category"] = df["product_rating"] - df["category_avg_rating"]

print("Most competitive category:", category_counts.idxmax())
print("Least competitive category:", category_counts.idxmin())


📌 WHY THIS STEP MATTERS:
Category-based features help us understand product competition and how well a product performs
compared to its peers. This is crucial for understanding market structure and identifying standout products.

🔍 WHAT WE'RE DOING:
- Calculating category_competition: number of products in each category.
- Computing category_avg_rating to measure category quality.
- Creating rating_vs_category to show how each product ranks relative to its peers.

📈 HOW TO INTERPRET THE OUTPUT:
• The most competitive category has the highest product count — likely a saturated market.
• The least competitive category may reflect niche or specialized items.
• rating_vs_category > 0 means the product is above average for its category.
• Large gaps indicate products that strongly overperform or underperform relative to peers.


9. CATEGORY FEATURES
----------------------------------------------------------------------
Most competitive category: Other Electronics
Least competitive category

In [65]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Interaction features capture relationships between variables that individual metrics cannot express.")
print("These help models detect combined effects of rating, price, discount, and brand strength.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Creating value_score (rating per dollar): measures value for money.")
print("- Creating deal_quality (discount × rating): captures how attractive the offer is.")
print("- Detecting premium brands based on brand frequency (top 10%).")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• Higher value_score means very strong rating for the price (great deals).")
print("• Higher deal_quality highlights products that are discounted *and* highly rated.")
print("• Premium brand count shows how many brands dominate the marketplace.")
print("• These features often improve ML model accuracy by capturing complex patterns.")
print("")
# ============================================================
# 10. INTERACTION FEATURES
# ============================================================
print("\n10. INTERACTION FEATURES")
print("-"*70)

df["value_score"] = df["product_rating"] / df["discounted_price"].replace(0, 1)
df["deal_quality"] = df["discount_percentage"] * df["product_rating"]

# Premium brand flag = top 10% most frequent brands
brand_freq_cutoff = df["brand"].value_counts().quantile(0.90)
top_brands = df["brand"].value_counts()[df["brand"].value_counts() >= brand_freq_cutoff].index
df["is_premium_brand"] = df["brand"].isin(top_brands).astype(int)

print("Premium brand count:", df["is_premium_brand"].sum())
print(f"Avg value score: {df['value_score'].mean():.4f}")


📌 WHY THIS STEP MATTERS:
Interaction features capture relationships between variables that individual metrics cannot express.
These help models detect combined effects of rating, price, discount, and brand strength.

🔍 WHAT WE'RE DOING:
- Creating value_score (rating per dollar): measures value for money.
- Creating deal_quality (discount × rating): captures how attractive the offer is.
- Detecting premium brands based on brand frequency (top 10%).

📈 HOW TO INTERPRET THE OUTPUT:
• Higher value_score means very strong rating for the price (great deals).
• Higher deal_quality highlights products that are discounted *and* highly rated.
• Premium brand count shows how many brands dominate the marketplace.
• These features often improve ML model accuracy by capturing complex patterns.


10. INTERACTION FEATURES
----------------------------------------------------------------------
Premium brand count: 25930
Avg value score: 0.1053


In [66]:
print("\n📌 WHY THIS STEP MATTERS:")
print("Outliers — extremely high prices or massive review counts — can distort model training, inflate variance,")
print("and skew distributions. Capping them ensures that ML models remain stable and resistant to noise.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Using percentile-based capping (1st–99th percentiles) to handle extreme values.")
print("- Applying it to total_reviews and discounted_price.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• After capping, the dataset becomes less skewed and more manageable.")
print("• This improves model convergence and reduces sensitivity to extreme values.")
print("• Outlier handling prevents single products from overly influencing predictions.")
print("")
# ============================================================
# 11. OUTLIER HANDLING
# ============================================================
print("\n11. OUTLIER HANDLING")
print("-"*70)

def cap_outliers(series, lower=0.01, upper=0.99):
    """Cap values at the 1st and 99th percentiles to prevent extreme skew."""
    return series.clip(series.quantile(lower), series.quantile(upper))

df["total_reviews_capped"] = cap_outliers(df["total_reviews"])
df["discounted_price_capped"] = cap_outliers(df["discounted_price"])

print("Outliers capped for: total_reviews, discounted_price")



📌 WHY THIS STEP MATTERS:
Outliers — extremely high prices or massive review counts — can distort model training, inflate variance,
and skew distributions. Capping them ensures that ML models remain stable and resistant to noise.

🔍 WHAT WE'RE DOING:
- Using percentile-based capping (1st–99th percentiles) to handle extreme values.
- Applying it to total_reviews and discounted_price.

📈 HOW TO INTERPRET THE OUTPUT:
• After capping, the dataset becomes less skewed and more manageable.
• This improves model convergence and reduces sensitivity to extreme values.
• Outlier handling prevents single products from overly influencing predictions.


11. OUTLIER HANDLING
----------------------------------------------------------------------
Outliers capped for: total_reviews, discounted_price


In [67]:
print("\n📌 WHY THIS STEP MATTERS:")
print("This summary verifies that all feature engineering steps completed successfully and gives us a final")
print("snapshot of the dataset. It ensures readiness for EDA, modeling, or visualization.")

print("\n🔍 WHAT WE'RE DOING:")
print("- Reporting final dataset shape after all transformations.")
print("- Listing key engineered features to verify completeness.")
print("- Showing a quick sample of important columns.")

print("\n📈 HOW TO INTERPRET THE OUTPUT:")
print("• A larger number of features means richer signals for ML models.")
print("• The sample displayed helps verify that features were created correctly.")
print("• This final check confirms that the dataset is clean, engineered, and ready for analysis.")
print("")
# ============================================================
# 12. FINAL SUMMARY
# ============================================================
print("\n12. FEATURE ENGINEERING SUMMARY")
print("="*70)

print(f"Final dataset shape: {df.shape}")

print("\nSample of final features:\n")
sample_cols = [
    "product_title", "brand", "product_rating", "total_reviews",
    "discounted_price", "discount_percentage", "trust_score", "review_score"
]
print(df[sample_cols].head())



📌 WHY THIS STEP MATTERS:
This summary verifies that all feature engineering steps completed successfully and gives us a final
snapshot of the dataset. It ensures readiness for EDA, modeling, or visualization.

🔍 WHAT WE'RE DOING:
- Reporting final dataset shape after all transformations.
- Listing key engineered features to verify completeness.
- Showing a quick sample of important columns.

📈 HOW TO INTERPRET THE OUTPUT:
• A larger number of features means richer signals for ML models.
• The sample displayed helps verify that features were created correctly.
• This final check confirms that the dataset is clean, engineered, and ready for analysis.


12. FEATURE ENGINEERING SUMMARY
Final dataset shape: (37718, 44)

Sample of final features:

                                       product_title  brand  product_rating  \
0  BOYA BOYALINK 2 Wireless Lavalier Microphone f...   boya             4.6   
1  LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...  lisen             4.3   
2  DJI Mic